In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("spam_utf8.csv")
df

,label,Text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,notspam,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,notspam,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
2,notspam,Ok lar... Joking wif u oni...,NaN,NaN,NaN
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
4,notspam,NaN,NaN,NaN,NaN
...,...,...,...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5570,notspam,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5571,notspam,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5572,notspam,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [9]:
#  Number of spam vs non spam emails
num_spam = df[df["label"]=="spam"].shape[0]
num_notspam = df[df["label"]=="notspam"].shape[0]

print(num_spam)
print(num_notspam)

747
4827


In [10]:
# Missing values count in dataset

missing_values = df["Text"].isnull().sum()
print(missing_values)

2
